# Exploración de datos con Python - datos del mundo real

En el último cuaderno, observamos las calificaciones de los datos de nuestros estudiantes e investigamos los datos visualmente con histogramas y diagramas de caja. Ahora analizaremos casos más complejos, describiremos los datos de manera más completa y discutiremos cómo hacer comparaciones básicas entre los datos.

### Distribuciones de datos en el mundo real

Anteriormente, analizamos las calificaciones para los datos de nuestros estudiantes y estimamos a partir de esta muestra cómo podría ser la población completa de calificaciones. Vamos a refrescar nuestra memoria y echar un vistazo a estos datos de nuevo.

Ejecute el siguiente código para imprimir los datos y hacer un histograma + diagrama de caja que muestre las calificaciones de nuestra muestra de estudiantes.

In [3]:
import pandas as pd
from matplotlib import pyplot as plt

# Load data from a text file
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/ml-basics/grades.csv
df_students = pd.read_csv('grades.csv',delimiter=',',header='infer')

# Remove any rows with missing data
df_students = df_students.dropna(axis=0, how='any')

# Calculate who passed, assuming '60' is the grade needed to pass
passes  = pd.Series(df_students['Grade'] >= 60)

# Save who passed to the Pandas dataframe
df_students = pd.concat([df_students, passes.rename("Pass")], axis=1)


# Print the result out into this notebook
print(df_students)


# Create a function that we can re-use
def show_distribution(var_data):
    '''
    This function will make a distribution (graph) and display it
    '''

    # Get statistics
    min_val = var_data.min()
    max_val = var_data.max()
    mean_val = var_data.mean()
    med_val = var_data.median()
    mod_val = var_data.mode()[0]

    print('Minimum:{:.2f}\nMean:{:.2f}\nMedian:{:.2f}\nMode:{:.2f}\nMaximum:{:.2f}\n'.format(min_val,
                                                                                            mean_val,
                                                                                            med_val,
                                                                                            mod_val,
                                                                                            max_val))

    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize = (10,4))

    # Plot the histogram   
    ax[0].hist(var_data)
    ax[0].set_ylabel('Frequency')

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mean_val, color = 'cyan', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=med_val, color = 'red', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mod_val, color = 'yellow', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=max_val, color = 'gray', linestyle='dashed', linewidth = 2)

    # Plot the boxplot   
    ax[1].boxplot(var_data, vert=False)
    ax[1].set_xlabel('Value')

    # Add a title to the Figure
    fig.suptitle('Data Distribution')

    # Show the figure
    fig.show()


show_distribution(df_students['Grade'])

--2023-03-30 00:07:17--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/ml-basics/grades.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322 [text/plain]
grades.csv: Permission denied

Cannot write to ‘grades.csv’ (Success).


FileNotFoundError: [Errno 2] No such file or directory: 'grades.csv'

Como recordarán, nuestros datos tenían la media y el modo en el centro, con datos distribuidos simétricamente desde allí.

Ahora echemos un vistazo a la distribución de los datos de horas de estudio.

In [ ]:
# Get the variable to examine
col = df_students['StudyHours']
# Call the function
show_distribution(col)

La distribución de los datos del tiempo de estudio es significativamente diferente de la de las calificaciones.

Tenga en cuenta que los bigotes del diagrama de caja solo comienzan alrededor de 6.0, lo que indica que la gran mayoría del primer trimestre de los datos está por encima de este valor. El mínimo está marcado con una o, lo que indica que estadísticamente es un valor atípico, un valor que se encuentra significativamente fuera del rango del resto de la distribución.

Los valores atípicos pueden ocurrir por muchas razones. Tal vez un estudiante tenía la intención de registrar "10" horas de tiempo de estudio, pero ingresó "1" y perdió el "0". ¡O tal vez el estudiante era anormalmente perezoso cuando se trata de estudiar! De cualquier manera, es una anomalía estadística que no representa a un estudiante típico. Veamos cómo se ve la distribución sin él.

In [ ]:
# Get the variable to examine
# We will only get students who have studied more than one hour
col = df_students[df_students.StudyHours>1]['StudyHours']

# Call the function
show_distribution(col)

Para fines de aprendizaje, acabamos de tratar el valor 1 como un verdadero valor atípico aquí y lo hemos excluido. En el mundo real, sería inusual excluir datos en los extremos sin más justificación cuando el tamaño de nuestra muestra es tan pequeño. Esto se debe a que cuanto menor sea el tamaño de nuestra muestra, más probable es que nuestro muestreo sea una mala representación de toda la población. (Aquí, la población significa calificaciones para todos los estudiantes, no solo para nuestros 22). Por ejemplo, si tomamos muestras del tiempo de estudio de otros 1,000 estudiantes, ¡podríamos encontrar que en realidad es bastante común no estudiar mucho!

Cuando tenemos más datos disponibles, nuestra muestra se vuelve más confiable. Esto hace que sea más fácil considerar los valores atípicos como valores que caen por debajo o por encima de los percentiles dentro de los cuales se encuentran la mayoría de los datos. Por ejemplo, el siguiente código utiliza la función cuantil de Pandas para excluir observaciones por debajo del percentil 0,01 (el valor por encima del cual reside el 99% de los datos).

In [ ]:
# calculate the 0.01th percentile
q01 = df_students.StudyHours.quantile(0.01)
# Get the variable to examine
col = df_students[df_students.StudyHours>q01]['StudyHours']
# Call the function
show_distribution(col)

> **Consejo**: También puede eliminar valores atípicos en el extremo superior de la distribución definiendo un umbral con un valor de percentil alto. Por ejemplo, podría usar la función cuantil para encontrar el percentil 0.99 por debajo del cual reside el 99% de los datos.

Con los valores atípicos eliminados, el diagrama de caja muestra todos los datos dentro de los cuatro cuartiles. Tenga en cuenta que la distribución no es simétrica como lo es para los datos de grado. Hay algunos estudiantes con tiempos de estudio muy altos de alrededor de 16 horas, pero la mayor parte de los datos es entre 7 y 13 horas. Los pocos valores extremadamente altos tiran de la media hacia el extremo superior de la escala.

Veamos la densidad de esta distribución.

In [ ]:
def show_density(var_data):
    fig = plt.figure(figsize=(10,4))

    # Plot density
    var_data.plot.density()

    # Add titles and labels
    plt.title('Data Density')

    # Show the mean, median, and mode
    plt.axvline(x=var_data.mean(), color = 'cyan', linestyle='dashed', linewidth = 2)
    plt.axvline(x=var_data.median(), color = 'red', linestyle='dashed', linewidth = 2)
    plt.axvline(x=var_data.mode()[0], color = 'yellow', linestyle='dashed', linewidth = 2)

    # Show the figure
    plt.show()

# Get the density of StudyHours
show_density(col)

Este tipo de distribución se llama sesgada a la derecha. La masa de los datos está en el lado izquierdo de la distribución, creando una larga cola hacia la derecha debido a los valores en el extremo superior, que tiran de la media hacia la derecha.

#### Medidas de varianza

Así que ahora tenemos una buena idea de dónde están las distribuciones de datos de la mitad del grado y las horas de estudio. Sin embargo, hay otro aspecto de las distribuciones que debemos examinar: ¿cuánta variabilidad hay en los datos?

Las estadísticas típicas que miden la variabilidad en los datos incluyen:

- **Rango**: La diferencia entre el máximo y el mínimo. No hay una función incorporada para esto, pero es fácil de calcular usando las funciones **min** y **max**.
- **Varianza**: El promedio de la diferencia al cuadrado de la media. Puede usar la función **var** incorporada para encontrar esto.
- **Desviación estándar**: La raíz cuadrada de la varianza. Puede usar la función **std** incorporada para encontrar esto.

In [ ]:
for col_name in ['Grade','StudyHours']:
    col = df_students[col_name]
    rng = col.max() - col.min()
    var = col.var()
    std = col.std()
    print('\n{}:\n - Range: {:.2f}\n - Variance: {:.2f}\n - Std.Dev: {:.2f}'.format(col_name, rng, var, std))

De estas estadísticas, la desviación estándar es generalmente la más útil. Proporciona una medida de varianza en los datos en la misma escala que los datos en sí (por lo tanto, puntos de calificación para la distribución de calificaciones y horas para la distribución de horas de estudio). Cuanto mayor es la desviación estándar, más varianza hay cuando se comparan los valores de la distribución con la media de la distribución; en otras palabras, los datos están más dispersos.

Cuando se trabaja con una distribución normal, la desviación estándar funciona con las características particulares de una distribución normal para proporcionar una visión aún mayor. Ejecute la celda siguiente para ver la relación entre las desviaciones estándar y los datos en la distribución normal.

In [ ]:
import scipy.stats as stats

# Get the Grade column
col = df_students['Grade']

# get the density
density = stats.gaussian_kde(col)

# Plot the density
col.plot.density()

# Get the mean and standard deviation
s = col.std()
m = col.mean()

# Annotate 1 stdev
x1 = [m-s, m+s]
y1 = density(x1)
plt.plot(x1,y1, color='magenta')
plt.annotate('1 std (68.26%)', (x1[1],y1[1]))

# Annotate 2 stdevs
x2 = [m-(s*2), m+(s*2)]
y2 = density(x2)
plt.plot(x2,y2, color='green')
plt.annotate('2 std (95.45%)', (x2[1],y2[1]))

# Annotate 3 stdevs
x3 = [m-(s*3), m+(s*3)]
y3 = density(x3)
plt.plot(x3,y3, color='orange')
plt.annotate('3 std (99.73%)', (x3[1],y3[1]))

# Show the location of the mean
plt.axvline(col.mean(), color='cyan', linestyle='dashed', linewidth=1)

plt.axis('off')

plt.show()

Las líneas horizontales muestran el porcentaje de datos dentro de 1, 2 y 3 desviaciones estándar de la media (más o menos).

En cualquier distribución normal:
- Aproximadamente el 68,26% de los valores caen dentro de una desviación estándar de la media.
- Aproximadamente el 95,45% de los valores caen dentro de dos desviaciones estándar de la media.
- Aproximadamente el 99,73% de los valores se encuentran dentro de tres desviaciones estándar de la media.

Entonces, como sabemos que la nota media es de 49,18, la desviación estándar es de 21,74 y la distribución de las notas es aproximadamente normal, podemos calcular que el 68,26% de los estudiantes deberían alcanzar una nota entre 27,44 y 70,92.

La estadística descriptiva que hemos utilizado para entender la distribución de las variables de datos de los estudiantes son la base del análisis estadístico. Debido a que son una parte tan importante de la exploración de los datos, hay un método integrado del objeto DataFrame que devuelve las estadísticas descriptivas principales para todas las columnas numéricas.describe

In [ ]:
df_students.describe()

## Comparación de datos

Ahora que sabe algo sobre la distribución estadística de los datos en su conjunto de datos, está listo para examinar sus datos para identificar cualquier relación aparente entre variables.

En primer lugar, vamos a deshacernos de cualquier fila que contenga valores atípicos para que tengamos una muestra que sea representativa de una clase típica de estudiantes. Identificamos que la columna StudyHours contiene algunos valores atípicos con valores extremadamente bajos, por lo que eliminaremos esas filas.

In [ ]:
df_sample = df_students[df_students['StudyHours']>1]
df_sample

### Comparación de variables numéricas y categóricas

Los datos incluyen dos variables numéricas (**StudyHours** y **Grade**) y dos variables categóricas (**Name** y **Pass**). Comencemos comparando la columna numérica **StudyHours** con la columna categórica **Pass** para ver si existe una relación aparente entre el número de horas estudiadas y una calificación aprobatoria.

Para hacer esta comparación, vamos a crear diagramas de caja que muestren la distribución de StudyHours para cada posible valor de Pass (verdadero y falso).

To make this comparison, let's create box plots showing the distribution of StudyHours for each possible Pass value (true and false).

In [ ]:
df_sample.boxplot(column='StudyHours', by='Pass', figsize=(8,5))

Comparando las distribuciones de StudyHours, es inmediatamente evidente (si no particularmente sorprendente) que los estudiantes que aprobaron el curso tendían a estudiar más horas que los estudiantes que no lo hicieron. Entonces, si desea predecir si es probable que un estudiante apruebe el curso, la cantidad de tiempo que pasa estudiando puede ser un buen indicador predictivo.

### Comparación de variables numéricas

Ahora comparemos dos variables numéricas. Comenzaremos creando un gráfico de barras que muestre tanto las calificaciones como las horas de estudio.

In [ ]:
# Create a bar plot of name vs grade and study hours
df_sample.plot(x='Name', y=['Grade','StudyHours'], kind='bar', figsize=(8,5))

El gráfico muestra barras para las horas de grado y estudio para cada estudiante, pero no es fácil de comparar porque los valores están en diferentes escalas. Una calificación se mide en puntos de calificación (y varía de 3 a 97), y el tiempo de estudio se mide en horas (y varía de 1 a 16).

Una técnica común cuando se trata de datos numéricos en diferentes escalas es normalizar los datos para que los valores conserven su distribución proporcional pero se midan en la misma escala. Para lograr esto, usaremos una técnica llamada escala MinMax que distribuye los valores proporcionalmente en una escala de 0 a 1. Puede escribir el código para aplicar esta transformación, pero la biblioteca Scikit-Learn proporciona un escalador para hacerlo por usted.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Get a scaler object
scaler = MinMaxScaler()

# Create a new dataframe for the scaled values
df_normalized = df_sample[['Name', 'Grade', 'StudyHours']].copy()

# Normalize the numeric columns
df_normalized[['Grade','StudyHours']] = scaler.fit_transform(df_normalized[['Grade','StudyHours']])

# Plot the normalized values
df_normalized.plot(x='Name', y=['Grade','StudyHours'], kind='bar', figsize=(8,5))

Con los datos normalizados, es más fácil ver una relación aparente entre el grado y el tiempo de estudio. No es una coincidencia exacta, pero definitivamente parece que los estudiantes con calificaciones más altas tienden a haber estudiado más.

Así que parece haber una correlación entre el tiempo de estudio y el grado. De hecho, hay una medida de **correlación** estadística que podemos usar para cuantificar la relación entre estas columnas.

In [ ]:
df_normalized.Grade.corr(df_normalized.StudyHours)

La estadística de correlación es un valor entre -1 y 1 que indica la fuerza de una relación. Los valores superiores a 0 indican una correlación positiva (los valores altos de una variable tienden a coincidir con los valores altos de la otra), mientras que los valores inferiores a 0 indican una correlación negativa (los valores altos de una variable tienden a coincidir con valores bajos de la otra). En este caso, el valor de correlación es cercano a 1, mostrando una correlación fuertemente positiva entre el tiempo de estudio y el grado.

> **Nota**: Los científicos de datos a menudo citan la máxima "correlación no es causalidad". En otras palabras, por muy tentador que sea, no debe interpretar la correlación estadística como una explicación de por qué uno de los valores es alto. En el caso de los datos de los estudiantes, las estadísticas demuestran que los estudiantes con calificaciones altas tienden a tener también altas cantidades de tiempo de estudio, pero esto no es lo mismo que demostrar que lograron altas calificaciones porque estudiaron mucho. La estadística también podría usarse como evidencia para apoyar la conclusión sin sentido de que los estudiantes estudiaron mucho porque sus calificaciones iban a ser altas.

Otra forma de visualizar la correlación aparente entre dos columnas numéricas es usar un diagrama de dispersión.

In [ ]:
# Create a scatter plot
df_sample.plot.scatter(title='Study Time vs Grade', x='StudyHours', y='Grade')

Una vez más, parece que hay un patrón discernible en el que los estudiantes que estudiaron la mayor cantidad de horas son también los estudiantes que obtuvieron las calificaciones más altas.

Podemos ver esto más claramente agregando una línea de regresión (o una línea de mejor ajuste) al gráfico que muestra la tendencia general en los datos. Para ello, utilizaremos una técnica estadística llamada regresión de mínimos cuadrados.

Recuerda cuando estabas aprendiendo a resolver ecuaciones lineales en la escuela, y recuerda que la forma de pendiente-intersección de una ecuación lineal se ve así:

$ y = mx + b $

En esta ecuación, y y x son las variables de coordenadas, m es la pendiente de la línea y b es la intersección y (donde la línea pasa por el eje Y).

En el caso de nuestro diagrama de dispersión para nuestros datos de estudiantes, ya tenemos nuestros valores para x (StudyHours) e y (Grade), por lo que solo necesitamos calcular la intersección y la pendiente de la línea recta que se encuentra más cerca de esos puntos. Entonces podemos formar una ecuación lineal que calcula un nuevo valor y en esa línea para cada uno de nuestros valores x (StudyHours). Para evitar confusiones, llamaremos a este nuevo valor y f(x) (porque es la salida de una ecuación lineal función basada en x). La diferencia entre el valor original y (Nota) y el valor f(x) es el error entre nuestra línea de regresión y la Nota real alcanzada por el estudiante. Nuestro objetivo es calcular la pendiente e interceptar una línea con el error general más bajo.

Específicamente, definimos el error general tomando el error para cada punto, cuadrándolo y sumando todos los errores al cuadrado. La línea de mejor ajuste es la línea que nos da el valor más bajo para la suma de los errores al cuadrado, de ahí el nombre de regresión de mínimos cuadrados.

Afortunadamente, no es necesario codificar el cálculo de regresión usted mismo. El paquete SciPy incluye una clase stats que proporciona un método linregress para hacer el trabajo duro por usted. Esto devuelve (entre otras cosas) los coeficientes que necesita para la ecuación de pendiente: pendiente (m) e intersección (b) en función de un par dado de muestras variables que desea comparar.

In [ ]:
from scipy import stats

#
df_regression = df_sample[['Grade', 'StudyHours']].copy()

# Get the regression slope and intercept
m, b, r, p, se = stats.linregress(df_regression['StudyHours'], df_regression['Grade'])
print('slope: {:.4f}\ny-intercept: {:.4f}'.format(m,b))
print('so...\n f(x) = {:.4f}x + {:.4f}'.format(m,b))

# Use the function (mx + b) to calculate f(x) for each x (StudyHours) value
df_regression['fx'] = (m * df_regression['StudyHours']) + b

# Calculate the error between f(x) and the actual y (Grade) value
df_regression['error'] = df_regression['fx'] - df_regression['Grade']

# Create a scatter plot of Grade vs StudyHours
df_regression.plot.scatter(x='StudyHours', y='Grade')

# Plot the regression line
plt.plot(df_regression['StudyHours'],df_regression['fx'], color='cyan')

# Display the plot
plt.show()

Tenga en cuenta que esta vez, el código trazó dos cosas distintas: el gráfico de dispersión de las horas y calificaciones de estudio de muestra se traza como antes, y luego se traza una línea de mejor ajuste basada en los coeficientes de regresión de mínimos cuadrados.

Los coeficientes de pendiente e intersección calculados para la línea de regresión se muestran encima de la gráfica.

La línea se basa en los valores f(x) calculados para cada valor de StudyHours. Ejecute la celda siguiente para ver una tabla que incluya los siguientes valores:

- Las Horas de Estudio para cada estudiante.
- La calificación alcanzada por cada estudiante.
- El valor f(x) calculado utilizando los coeficientes de línea de regresión.
- Error entre el valor f(x) calculado y el valor real de Grade.

Algunos de los errores, particularmente en los extremos extremos, y bastante grandes (hasta más de 17.5 puntos de calificación). Pero, en general, la línea está bastante cerca de las calificaciones reales.

In [ ]:
# Show the original x,y values, the f(x) value, and the error
df_regression[['StudyHours', 'Grade', 'fx', 'error']]

### Uso de los coeficientes de regresión para la predicción

Ahora que tiene los coeficientes de regresión para la relación entre el tiempo de estudio y la calificación, puede usarlos en una función para estimar la calificación esperada para una cantidad determinada de estudio.

In [ ]:
# Define a function based on our regression coefficients
def f(x):
    m = 6.3134
    b = -17.9164
    return m*x + b

study_time = 14

# Get f(x) for study time
prediction = f(study_time)

# Grade can't be less than 0 or more than 100
expected_grade = max(0,min(100,prediction))

#Print the estimated grade
print ('Studying for {} hours per week may result in a grade of {:.0f}'.format(study_time, expected_grade))

Por lo tanto, al aplicar estadísticas a los datos de muestra, ha determinado una relación entre el tiempo de estudio y la calificación y ha encapsulado esa relación en una función general que se puede usar para predecir una calificación para una cantidad determinada de tiempo de estudio.

Esta técnica es, de hecho, la premisa básica del aprendizaje automático. Puede tomar un conjunto de datos de ejemplo que incluya una o más características (en este caso, el número de horas estudiadas) y un valor de etiqueta conocido (en este caso, la calificación obtenida) y utilizar los datos de muestra para derivar una función que calcule los valores de etiqueta previstos para cualquier conjunto determinado de características.
